## Speed test for loops and reshaped representations

We want to see which is the best shape for doing calculations conveniently and faster.

In [1]:
using Base.Threads
using Distributions
using LaTeXStrings
using Printf
using LsqFit
using BenchmarkTools
using LinearAlgebra

In [2]:
nthreads()

10

Typical reshape for a NxN grid and a 2-vector.

In [4]:
const D = 2
#const J1 = 1000
#const J2 = 1000


J = (1000,1000)

EE = rand(D*J[1]*J[2])
E = reshape(EE,(D,J...))

function prueba_vel_ji!(EE,D,J)
    @inbounds E = reshape(EE,(D,J...))
    @inbounds for j in 1:J[2]
        @inbounds for i in 1:J[1]
            E[1,i,j] = 5.0 * E[2,i,j]
        end
    end
end

function prueba_vel_threads_ji!(EE,D,J)
    @inbounds E = reshape(EE,(D,J...))
    @inbounds @threads for j in 1:J[2]
        @inbounds    for i in 1:J[1]
            E[1,i,j] = 5.0 * E[2,i,j]
        end
    end
end

function prueba_vel_ij!(EE,D,J)
    @inbounds E = reshape(EE,(D,J...))
    @inbounds for i in 1:J[1]
        @inbounds for j in 1:J[2]
            E[1,i,j] = 5.0 * E[2,i,j]
        end
    end
end

function prueba_vel_threads_ij!(EE,D,J)
    @inbounds E = reshape(EE,(D,J...))
    @inbounds for i in 1:J[1]
        @inbounds for j in 1:J[2]
            E[1,i,j] = 5.0 * E[2,i,j]
        end
    end
end

function prueba_vel_no_for!(EE,D,J)
    @inbounds E = reshape(EE,(D,J...))   
    @inbounds E[1,:,:] .= 5.0 * E[2,:,:]
    return 0
end

@show @btime prueba_vel_ij!($EE,$D,$J)
@show @btime prueba_vel_ji!($EE,$D,$J)
@show @btime prueba_vel_threads_ij!($EE,$D,$J)
@show @btime prueba_vel_threads_ji!($EE,$D,$J)
@show @btime prueba_vel_no_for!($EE,$D,$J)

#sum(reshape(EE,2,100,100)[1,:,:] - 5*reshape(EE,2,100,100)[2,:,:])


  699.251 μs (2 allocations: 96 bytes)
#= /users/oreula/Julia/PIC-1D/Tests/speed_test.ipynb:53 =# @btime(prueba_vel_ij!($(Expr(:$, :EE)), $(Expr(:$, :D)), $(Expr(:$, :J)))) = nothing
  

320.905 μs (2 allocations: 96 bytes)
#= /users/oreula/Julia/PIC-1D/Tests/speed_test.ipynb:54 =# @btime(prueba_vel_ji!($(Expr(:$, :EE)), $(Expr(:$, :D)), $(Expr(:$, :J)))) = nothing


  695.642 μs (2 allocations: 96 bytes)
#= /users/oreula/Julia/PIC-1D/Tests/speed_test.ipynb:55 =# @btime(prueba_vel_threads_ij!($(Expr(:$, :EE)), $(Expr(:$, :D)), $(Expr(:$, :J)))) = nothing


  122.846 μs (63 allocations: 5.53 KiB)
#= /users/oreula/Julia/PIC-1D/Tests/speed_test.ipynb:56 =# @btime(prueba_vel_threads_ji!($(Expr(:$, :EE)), $(Expr(:$, :D)), $(Expr(:$, :J)))) = nothing
  

1.653 ms (6 allocations: 15.26 MiB)
#= /users/oreula/Julia/PIC-1D/Tests/speed_test.ipynb:57 =# @btime(prueba_vel_no_for!($(Expr(:$, :EE)), $(Expr(:$, :D)), $(Expr(:$, :J)))) = 0


0

### 100x100

12.786 μs (14 allocations: 1.09 KiB) @btime(prueba_vel_ij!($(Expr(:$, :EE))))  **con threads(2)**

3.916 μs (2 allocations: 96 bytes) @btime(prueba_vel_ji!($(Expr(:$, :EE)))) = nothing

23.232 μs (6 allocations: 156.44 KiB) @btime(prueba_vel_no_for!($(Expr(:$, :EE)))) = 0

### 1000x1000 

1.297 ms (14 allocations: 1.09 KiB) @btime(prueba_vel_ij!($(Expr(:$, :EE)))) **con threads(2)** 

2.363 ms (2 allocations: 96 bytes) @btime(prueba_vel_ij!($(Expr(:$, :EE)))) = nothing

534.565 μs (14 allocations: 1.09 KiB) @btime(prueba_vel_ji!($(Expr(:$, :EE)))) = nothing **con threads(2)**

819.136 μs (2 allocations: 96 bytes) @btime(prueba_vel_ji!($(Expr(:$, :EE)))) = nothing

3.745 ms (6 allocations: 15.26 MiB) @btime(prueba_vel_no_for!($(Expr(:$, :EE)))) = 0

**clearly the fast indices are j,i**

**In ATOM**

699.251 μs (2 allocations: 96 bytes)
@btime(prueba_vel_ij!($(Expr(:$, :EE)), $(Expr(:$, :D)), $(Expr(:$, :J)))) = nothing
  
320.905 μs (2 allocations: 96 bytes) 
@btime(prueba_vel_ji!($(Expr(:$, :EE)), $(Expr(:$, :D)), $(Expr(:$, :J)))) = nothing

695.642 μs (2 allocations: 96 bytes)
@btime(prueba_vel_threads_ij!($(Expr(:$, :EE)), $(Expr(:$, :D)), $(Expr(:$, :J)))) = nothing

122.846 μs (63 allocations: 5.53 KiB)
@btime(prueba_vel_threads_ji!($(Expr(:$, :EE)), $(Expr(:$, :D)), $(Expr(:$, :J)))) = nothing

1.653 ms (6 allocations: 15.26 MiB)
@btime(prueba_vel_no_for!($(Expr(:$, :EE)), $(Expr(:$, :D)), $(Expr(:$, :J)))) = 0
0


Checking that the values are written.

In [6]:
sum(reshape(EE,(D,J...))[1,:,:] - 5*reshape(EE,(D,J...))[2,:,:])

0.0

### With the vector index at the end

In [20]:
function prueba_2_vel_ji!(EE,D,J)
    @inbounds E = reshape(EE,(J...,D))
    @inbounds for j in 1:J[2]
        @inbounds for i in 1:J[1]
            E[i,j,1] = 5.0 * E[i,j,2]
        end
    end
end

function prueba_2_vel_ij_th!(EE,D,J)
    @inbounds E = reshape(EE,(J...,D))
    @inbounds  @threads  for i in 1:J[1]
        @inbounds    for j in 1:J[2]
            E[i,j,1] = 5.0 * E[i,j,1]
        end
    end
end

function prueba_2_vel_ji_th!(EE,D,J)
    @inbounds E = reshape(EE,(J...,D))
    @inbounds @threads for j in 1:J[2]
        @inbounds for i in 1:J[1]
            E[i,j,1] = 5.0 * E[i,j,2]
        end
    end
end

function prueba_2_vel_ij!(EE,D,J)
    @inbounds E = reshape(EE,(J...,D))
    @inbounds  for i in 1:J[1]
        @inbounds    for j in 1:J[2]
            E[i,j,1] = 5.0 * E[i,j,1]
        end
    end
end

function prueba_2_vel_no_for!(EE,D,J)
    @inbounds E = reshape(EE,(J...,D))   
    @inbounds E[:,:,1] .= 5.0 .* @view E[:,:,2]
    return 0
end

@show @btime prueba_2_vel_ij!($EE,D,J)
@show @btime prueba_2_vel_ij_th!($EE,D,J)
@show @btime prueba_2_vel_ji!($EE,D,J)
@show @btime prueba_2_vel_ji_th!($EE,D,J)
@show @btime prueba_2_vel_no_for!($EE,D,J)

  619.997 μs (2 allocations: 96 bytes)
#= /users/oreula/Julia/PIC-1D/Tests/speed_test.ipynb:43 =# @btime(prueba_2_vel_ij!($(Expr(:$, :EE)), D, J)) = nothing


  112.065 μs (63 allocations: 5.53 KiB)
#= /users/oreula/Julia/PIC-1D/Tests/speed_test.ipynb:44 =# @btime(prueba_2_vel_ij_th!($(Expr(:$, :EE)), D, J)) = nothing
  

254.162 μs (2 allocations: 96 bytes)
#= /users/oreula/Julia/PIC-1D/Tests/speed_test.ipynb:45 =# @btime(prueba_2_vel_ji!($(Expr(:$, :EE)), D, J)) = nothing


  56.252 μs (63 allocations: 5.53 KiB)
#= /users/oreula/Julia/PIC-1D/Tests/speed_test.ipynb:46 =# @btime(prueba_2_vel_ji_th!($(Expr(:$, :EE)), D, J)) = nothing
  

189.809 μs (2 allocations: 96 bytes)
#= /users/oreula/Julia/PIC-1D/Tests/speed_test.ipynb:47 =# @btime(prueba_2_vel_no_for!($(Expr(:$, :EE)), D, J)) = 0


0

## 100x100

10.943 μs (14 allocations: 1.09 KiB) **con threads ij**

1.831 μs (2 allocations: 96 bytes) ji

23.328 μs (6 allocations: 156.44 KiB) ::,::

## 1000x1000

869.742 μs (14 allocations: 1.09 KiB) @btime(prueba_2_vel_ij!($(Expr(:$, :EE)))) = nothing **con threads**

572.861 μs (2 allocations: 96 bytes) @btime(prueba_2_vel_ji!($(Expr(:$, :EE)))) = nothing

1.360 ms (2 allocations: 96 bytes) @btime(prueba_2_vel_ij!($(Expr(:$, :EE)))) = nothing

398.227 μs (14 allocations: 1.09 KiB) @btime(prueba_2_vel_ji!($(Expr(:$, :EE)))) = nothing **con threads**
  
3.686 ms (6 allocations: 15.26 MiB) @btime(prueba_2_vel_no_for!($(Expr(:$, :EE)))) = 0

2.600 ms (4 allocations: 7.63 MiB) @btime(prueba_2_vel_no_for!($(Expr(:$, :EE)))) **con . en - y**

845.347 μs (2 allocations: 96 bytes) @btime(prueba_2_vel_no_for!($(Expr(:$, :EE)))) **con @view en el lado derecho**

**In ATOM**

612.167 μs (2 allocations: 96 bytes)
@btime(prueba_2_vel_ij!($(Expr(:$, :EE)), D, J)) = nothing

121.515 μs (63 allocations: 5.53 KiB)
@btime(prueba_2_vel_ij!($(Expr(:$, :EE)), D, J)) = nothing **threads**

55.882 μs (63 allocations: 5.53 KiB)
@btime(prueba_2_vel_ji!($(Expr(:$, :EE)), D, J)) = nothing **threads**

190.819 μs (2 allocations: 96 bytes)
@btime(prueba_2_vel_no_for!($(Expr(:$, :EE)), D, J)) = 0
0

In [ ]:
sum(reshape(EE,J1,J2,D)[:,:,1] - 5*reshape(EE,J1,J2,D)[:,:,2])

For a scalar field

In [15]:
#D = 1
#J1 = 100
#J2 = 100

J = (1000,1000)
BB = rand(J1*J2)

function prueba_scalar_ji!(BB,J)
    @inbounds B = reshape(BB,J[1],J[2])
    @inbounds  for j in 1:J[2]
        @inbounds for i in 1:J[1]
            B[i,j] = 5.0 * B[i,j]
        end
    end
end

function prueba_scalar_threads_ji!(BB,J)
    @inbounds B = reshape(BB,J[1],J[2])
    @inbounds  @threads for j in 1:J[2]
        @inbounds for i in 1:J[1]
            B[i,j] = 5.0 * B[i,j]
        end
    end
end

function prueba_scalar_ij!(BB,J)
    @inbounds B = reshape(BB,J[1],J[2])
    @inbounds for i in 1:J[1]
        @inbounds    for j in 1:J[2]
            B[i,j] = 5.0 * B[i,j]
        end
    end
end

function prueba_scalar_threads_ij!(BB,J)
    @inbounds B = reshape(BB,J[1],J[2])
    @inbounds @threads for i in 1:J[1]
        @inbounds    for j in 1:J[2]
            B[i,j] = 5.0 * B[i,j]
        end
    end
end

function prueba_scalar_no_for!(BB,J)
    @inbounds B = reshape(BB,J[1],J[2])   
    @inbounds B[:,:] .= 5.0 .* @view B[:,:]
    return 0
end

@show @btime prueba_scalar_ij!($BB,$J)
@show @btime prueba_scalar_ji!($BB,$J)
@show @btime prueba_scalar_threads_ij!($BB,$J)
@show @btime prueba_scalar_threads_ji!($BB,$J)
@show @btime prueba_scalar_no_for!($BB,$J)

  1.650 ms (2 allocations: 96 bytes)
#= /Users/reula/Julia/PIC/PIC-1D/Tests/speed_test.ipynb:50 =# @btime(prueba_scalar_ij!($(Expr(:$, :BB)), $(Expr(:$, :J)))) = nothing
  

327.403 μs (2 allocations: 96 bytes)
#= /Users/reula/Julia/PIC/PIC-1D/Tests/speed_test.ipynb:51 =# @btime(prueba_scalar_ji!($(Expr(:$, :BB)), $(Expr(:$, :J)))) = nothing


  856.828 μs (14 allocations: 1.12 KiB)
#= /Users/reula/Julia/PIC/PIC-1D/Tests/speed_test.ipynb:52 =# @btime(prueba_scalar_threads_ij!($(Expr(:$, :BB)), $(Expr(:$, :J)))) = nothing


  205.084 μs (14 allocations: 1.12 KiB)
#= /Users/reula/Julia/PIC/PIC-1D/Tests/speed_test.ipynb:53 =# @btime(prueba_scalar_threads_ji!($(Expr(:$, :BB)), $(Expr(:$, :J)))) = nothing
  

555.944 μs (2 allocations: 96 bytes)
#= /Users/reula/Julia/PIC/PIC-1D/Tests/speed_test.ipynb:54 =# @btime(prueba_scalar_no_for!($(Expr(:$, :BB)), $(Expr(:$, :J)))) = 0


0

## 100x100

**sin definir las constantes** 

578.613 μs (30117 allocations: 629.31 KiB) @btime(prueba_scalar_ij!($(Expr(:$, :BB)))) = nothing **threads**
  
1.036 ms (29903 allocations: 625.12 KiB) @btime(prueba_scalar_ji!($(Expr(:$, :BB)))) = nothing 

18.836 μs (14 allocations: 156.69 KiB) @btime(prueba_scalar_no_for!($(Expr(:$, :BB)))) = 0

**con las constantes**

11.930 μs (14 allocations: 1.09 KiB) @btime(prueba_scalar_ij!($(Expr(:$, :BB)))) = nothing **threads**

1.494 μs (2 allocations: 96 bytes) @btime(prueba_scalar_ji!($(Expr(:$, :BB)))) = nothing 

16.905 μs (6 allocations: 156.44 KiB) @btime(prueba_scalar_no_for!($(Expr(:$, :BB)))) = 0

3.792 μs (2 allocations: 96 bytes) @btime(prueba_scalar_ij!($(Expr(:$, :BB)))) = nothing

9.847 μs (14 allocations: 1.09 KiB) @btime(prueba_scalar_ji!($(Expr(:$, :BB)))) = nothing **threads**

**Nota: Las threads las ponemos siempre en el loop externo.**

## 1000x1000

1.875 ms (2 allocations: 96 bytes) @btime(prueba_scalar_ij!($(Expr(:$, :BB)))) = nothing

856.074 μs (14 allocations: 1.09 KiB) @btime(prueba_scalar_ij!($(Expr(:$, :BB)))) = nothing **threads**

318.992 μs (2 allocations: 96 bytes) @btime(prueba_scalar_ji!($(Expr(:$, :BB)))) = nothing

201.421 μs (14 allocations: 1.09 KiB) @btime(prueba_scalar_ji!($(Expr(:$, :BB)))) = nothing **threads**

3.088 ms (6 allocations: 15.26 MiB) @btime(prueba_scalar_no_for!($(Expr(:$, :BB))))

1.934 ms (4 allocations: 7.63 MiB) @btime(prueba_scalar_no_for!($(Expr(:$, :BB)))) **doble .**

556.480 μs (2 allocations: 96 bytes) @btime(prueba_scalar_no_for!($(Expr(:$, :BB)))) **con @view B del lado derecho**


## Memory alignment

Veamos como se alinea la memoria con vec. 

In [15]:
E_first = [(i-1)*2+(j-1)*6+l for l in 1:2, i in 1:3, j in 1:4]

2×3×4 Array{Int64, 3}:
[:, :, 1] =
 1  3  5
 2  4  6

[:, :, 2] =
 7   9  11
 8  10  12

[:, :, 3] =
 13  15  17
 14  16  18

[:, :, 4] =
 19  21  23
 20  22  24

In [16]:
vec(E_first)

24-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
  ⋮
 16
 17
 18
 19
 20
 21
 22
 23
 24

In [17]:
E_last = [(i-1)*2+(j-1)*6+l for i in 1:3, j in 1:4, l in 1:2]

3×4×2 Array{Int64, 3}:
[:, :, 1] =
 1   7  13  19
 3   9  15  21
 5  11  17  23

[:, :, 2] =
 2   8  14  20
 4  10  16  22
 6  12  18  24

In [18]:
vec(E_last)

24-element Vector{Int64}:
  1
  3
  5
  7
  9
 11
 13
 15
 17
 19
  ⋮
  8
 10
 12
 14
 16
 18
 20
 22
 24